In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import IPython.display as display

In [2]:
# setup latex formatting
from matplotlib import rc
rc('text', usetex=True)
rc('font', family='serif', size=28)
rc('axes', labelsize=28)

## Inputs

In [3]:
MAX_DIM = 512  # limit images to MAX_DIM x MAX_DIM resolution
MAX_RGB = 255  # maximum RGB intensize

VGG19_INPUT_SHAPE = (224, 224)  # shape of input taken by VV19

content_path = 'images/in/sybil.jpg'  # location of content image
style_path = 'images/in/cezanne_style.jpg'  # location of style image

# for matching features in generated image to content and style images
content_layers = ['block5_conv2']
style_layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1',
                'block4_conv1',
                'block5_conv1']

## Preprocessing

In [4]:
def load_img(path):
    """ Load and resize image.
    
    Load image from the specified path into a tensor and resize to
    safely propagate through network.
    
    Args:
        path (str): path to image
        
    Returns:
        img (tf.Tensor): tensor representing image with max size
            (1, MAX_DIM, MAX_DIM, 3)
    """
    img = tf.read_file(path)
    img = tf.cond(tf.image.is_jpeg(img),
                  lambda: tf.image.decode_jpeg(img, channels=3),
                  lambda: tf.image.decode_png(img, channels=3))
    img = tf.image.convert_image_dtype(img, tf.float32)
    # determine scaling to reduce to MAX_DIM
    orig_shape = tf.cast(tf.shape(img)[:-1], tf.float32)
    long_dim = tf.reduce_max(orig_shape)
    scale = MAX_DIM / long_dim
    # cast into new shape ()
    scaled_shape = tf.cast(orig_shape * scale, tf.int32)
    img = tf.image.resize(img, scaled_shape)
    img = img[tf.newaxis, :]

    return img

In [5]:
content_img = load_img(content_path)
style_img = load_img(style_path)

## Load pre-trained neural network

In [6]:
def vgg_intermediate_output(layer_names):
    """ Create pretrained VGG model with intermediate outputs.
    
    Load the pretrained VGG network and construct a model which outputs
    from intermediate layers.
    
    Args:
        layer_names (list): list of layer names to output from.
        
    Returns:
        model (tf.keras.Model): the corresponding model
    """
    # include_top=False -> no classification layer
    vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet')
    vgg.trainable = False
    
    outputs = [vgg.get_layer(name).output for name in layer_names]
    
    model = tf.keras.Model([vgg.input], outputs)
    return model

## Train model

In [7]:
style_model = vgg_intermediate_output(style_layers)
style_outputs = style_model(style_img*MAX_RGB)

W0710 12:14:20.048684 4583912896 deprecation.py:506] From /anaconda3/envs/tensorflow_cpu/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [8]:
def compute_gram_matrix(input_tensor):
    """ Compute the normalised Gram matrix.
    
    Takes a batch (b) of matrices with and computes the Gram matrix for each.
    
    Args:
        input_tensor (tf.Tensor): Tensor containing batch of matrices.
    """
    result = tf.linalg.einsum('bijc,bijd->bcd', input_tensor, input_tensor)
    input_shape = tf.shape(input_tensor)
    n_elems = tf.cast(input_shape[1]*input_shape[2], tf.float32)
    return result/n_elems

In [9]:
class StyleContentModel(tf.keras.models.Model):
    """ A model which returns the style and content tensors of an image.
    """
    def __init__(self, style_laters, content_layers):
        """ Create pretrained VGG model with intermediate layer outputs.
        
        Args:
            style_layers (list): Names of intermediate VGG layers to 
                extract style outputs from.
            content_layers (list): Names of intermediate VGG layers to 
                extract content outputs from.
        """
        super(StyleContentModel, self).__init__()
        self.vgg = vgg_intermediate_output(style_layers + content_layers)
        self.style_layers = style_layers
        self.content_layers = content_layers
        self.num_style_layers = len(style_layers)
        self.vgg.trainable = False
        
    def call(self, inputs):
        """ Extract the content and style outputs for a given input.
        
        Given an image, preprocess and pass to the pretrained VGG19 network. 
        Extract and neatly package the style and content outputs.
        
        Args:
            inputs (tf.Tensor): (Batch of) input image(s) with pixel values in
                range (0,1).
        
        Returns:
            cache (dict): Neatly packaged cache of style and content outputs.
        """
        inputs = inputs*MAX_RGB
        preprocessed_inputs = tf.keras.applications.vgg19.preprocess_input(inputs)
        outputs = self.vgg(preprocessed_inputs)
        style_outputs, content_outputs = (outputs[:self.num_style_layers],
                                          outputs[self.num_style_layers:])
        
        style_outputs = [compute_gram_matrix(style_output)
                         for style_output in style_outputs]
        
        content_dict = {content_name : value
                        for content_name, value
                        in zip(self.content_layers, content_outputs)}
        
        style_dict = {style_name : value
                      for style_name, value
                      in zip(self.style_layers, style_outputs)}
        
        cache = {'content' : content_dict, 'style' : style_dict} 
        return cache

In [10]:
extractor = StyleContentModel(style_layers, content_layers)
results = extractor(content_img)

In [59]:
style_content_loss(results)

<tf.Tensor 'add_11:0' shape=() dtype=float32>

In [11]:
style_targets = extractor(style_img)['style']
content_targets = extractor(content_img)['content']

In [26]:
def create_starting_img(content_img):
    img = tf.Variable(tf.random.uniform((1, VGG19_INPUT_SHAPE[0], VGG19_INPUT_SHAPE[1], 3)))
    return img

In [27]:
img = create_starting_img(content_img)

In [30]:
def clip(img, lo=0., hi=1.):
    return tf.clip_by_value(img, clip_value_min=lo, clip_value_max=hi)

In [54]:
opt = tf.train.AdamOptimizer(learning_rate=0.02, beta1=0.99, epsilon=1e-1).minimize(loss)

In [32]:
def style_content_loss(outputs, style_weight=1e-2, content_weight=1e5):
    """ Compute the weighted loss of the given outputs.
    """
    style_outputs = outputs['style']
    content_outputs = outputs['content']
    style_loss = tf.add_n([tf.reduce_mean((style_outputs[name] - style_targets[name])**2)
                           for name in style_outputs.keys()])
    style_loss *= style_weight / len(style_outputs)
    
    content_loss = tf.add_n([tf.reduce_mean((content_outputs[name] - content_targets[name])**2)
                             for name in content_outputs.keys()])
    content_loss *= content_weight / len(content_outputs)
    
    loss = style_loss + content_loss
    return loss

In [33]:
def high_pass_x_y(img):
    x_var = img[:,:,1:,:] - img[:,:,:-1,:]
    y_var = img[:,1:,:,:] - img[:,:-1,:,:]
    return x_var, y_var

def total_variational_loss(img):
    x_deltas, y_deltas = high_pass_x_y(img)
    return tf.reduce_mean(x_deltas**2) + tf.reduce_mean(y_deltas**2)

In [34]:
@tf.function()
def train_step(img, total_variational_weight=1e8):
    with tf.GradientTape() as tape:
        outputs = extractor(img)
        loss = style_content_loss(outputs)
        loss += total_variational_weight*total_variational_loss(img)
    
        grad = tape.gradient(loss, img)
        opt.apply_gradients([(grad, img)])
        img.assign(clip(img))

In [30]:
def display_and_save_images(content_img, combined_img, style_img, img_name=None, img_base='images/out/'):
    # plot
    fig, axs = plt.subplots(1, 3, figsize=(18,6))
    plot_img(axs[0], content_img, 'Content Image')
    plot_img(axs[1], combined_img, 'Combined')
    plot_img(axs[2], style_img, 'Style Image')
    # format
    fig.tight_layout()
    # save
    if img_name is None:
        img_path = img_base+"combined.png"
    else:
        img_path = img_base+img_name+".png"
    fig.savefig(img_path, bbox_inches='tight')
    plt.show()

In [63]:
img = tf.Variable(content_img)

ValueError: initial_value must have a shape specified: Tensor("strided_slice_1:0", shape=(1, ?, ?, 3), dtype=float32)

In [56]:
epochs = 1
steps_per_epoch = 10

In [57]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    step = 0
    for n in range(epochs):
        for m in range(steps_per_epoch):
            step += 1
            _, c = sess.run([opt, loss])
            print(".", end='')

InvalidArgumentError: Incompatible shapes: [1,14,14,512] vs. [1,32,32,512]
	 [[node gradients_7/sub_65_grad/BroadcastGradientArgs (defined at <ipython-input-54-89d529a9b26a>:1) ]]

Original stack trace for 'gradients_7/sub_65_grad/BroadcastGradientArgs':
  File "/anaconda3/envs/tensorflow_cpu/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/anaconda3/envs/tensorflow_cpu/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/anaconda3/envs/tensorflow_cpu/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/anaconda3/envs/tensorflow_cpu/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/anaconda3/envs/tensorflow_cpu/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/anaconda3/envs/tensorflow_cpu/lib/python3.6/site-packages/tornado/platform/

In [26]:
"""
fig, axs = plt.subplots(1, 3, figsize=(18,6))
axs[0].imshow(tf.squeeze(content_img, axis=0))
axs[1].imshow(tf.squeeze(style_img, axis=0))
axs[2].imshow(img.read_value()[0])
axs[0].axis('off')
axs[1].axis('off')
axs[2].axis('off')
axs[0].set_title('Content Image')
axs[1].set_title('Style Image')
axs[2].set_title('Combined')
"""

"\nfig, axs = plt.subplots(1, 3, figsize=(18,6))\naxs[0].imshow(tf.squeeze(content_img, axis=0))\naxs[1].imshow(tf.squeeze(style_img, axis=0))\naxs[2].imshow(img.read_value()[0])\naxs[0].axis('off')\naxs[1].axis('off')\naxs[2].axis('off')\naxs[0].set_title('Content Image')\naxs[1].set_title('Style Image')\naxs[2].set_title('Combined')\n"

In [45]:
import os
import imageio

In [47]:
images = []
for i in range(0,2001,50)[1:]:
    path = 'images/out/gif/combined_%d.png'%i
    images.append(imageio.imread(path))
for i in range(50):
    images.append(imageio.imread(path))
imageio.mimsave('images/out/process.gif', images)

In [54]:
mpimg.imsave('images/out/sybil_2000_steps.png', tf.squeeze(img, axis=0).numpy())

In [52]:
img

<tf.Variable 'Variable:0' shape=(1, 512, 512, 3) dtype=float32, numpy=
array([[[[0.4853375 , 0.4070687 , 0.21224578],
         [0.49325696, 0.40537363, 0.21416649],
         [0.51216877, 0.4325639 , 0.24133198],
         ...,
         [0.47865784, 0.5217105 , 0.21765208],
         [0.5125218 , 0.55106246, 0.2146284 ],
         [0.5138633 , 0.5584453 , 0.20674834]],

        [[0.47624263, 0.41051453, 0.21441986],
         [0.49277648, 0.4262772 , 0.23299034],
         [0.5095588 , 0.4595523 , 0.26200613],
         ...,
         [0.45522287, 0.5151871 , 0.23813616],
         [0.47685027, 0.5265361 , 0.21656656],
         [0.485423  , 0.5214637 , 0.1871035 ]],

        [[0.45382747, 0.43344113, 0.23642102],
         [0.46725896, 0.45505232, 0.26318938],
         [0.47213545, 0.4838665 , 0.2890054 ],
         ...,
         [0.43371195, 0.5052511 , 0.27371216],
         [0.44882444, 0.49998575, 0.223767  ],
         [0.448546  , 0.48441616, 0.19247967]],

        ...,

        [[0.72896683,